In [1]:
! pip install transformers numpy gensim requests nlpaug nltk albumentations -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.5 MB/s eta 0:00:00


In [2]:
# !gdown "0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t"

In [3]:
import re
import nltk
import random
import numpy as np
import pandas as pd
import transformers
from tqdm import tqdm
import multiprocessing as mp
from nltk import sent_tokenize
from albumentations.core.transforms_interface import DualTransform, BasicTransform

nltk.download('punkt')

In [4]:
!gdown 1o1VCMOLmx7nTcztg4IG65ed2c0oxPqG4 # train_ground_truth.csv
!gdown 1Il8FjSe-2aYg9NhCqG2yGFU7FtLDfKp4 # train_input_paragraph.csv
!gdown 1-h0dcPiuEiWoseXrjRhj2Ru3wv9-p7aL # train_input_question.csv
!gdown 1DszAla41KYcYn3vRsyb6pgTbguIVLVD8 # train_theme_interval.csv
!gdown 1Z-yb752A3o7b9dqrGt24XU0sl53FVqya # total train data

Downloading...
From: https://drive.google.com/uc?id=1o1VCMOLmx7nTcztg4IG65ed2c0oxPqG4
To: /content/train_ground_truth.csv
100% 2.23M/2.23M [00:00<00:00, 231MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Il8FjSe-2aYg9NhCqG2yGFU7FtLDfKp4
To: /content/train_input_paragraph.csv
100% 12.0M/12.0M [00:00<00:00, 161MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-h0dcPiuEiWoseXrjRhj2Ru3wv9-p7aL
To: /content/train_input_question.csv
100% 6.06M/6.06M [00:00<00:00, 227MB/s]
Downloading...
From: https://drive.google.com/uc?id=1DszAla41KYcYn3vRsyb6pgTbguIVLVD8
To: /content/train_theme_interval.csv
100% 9.88k/9.88k [00:00<00:00, 12.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Z-yb752A3o7b9dqrGt24XU0sl53FVqya
To: /content/train_data.csv
100% 65.6M/65.6M [00:00<00:00, 305MB/s]


In [5]:
train_df = pd.read_csv("train_data.csv")

In [6]:
train_df["Answer_text"] = train_df["Answer_text"].apply(lambda x: x[2:-2])
train_df["Answer_start"] = train_df["Answer_start"].apply(lambda x: int(x[1:-1]) if x != '[]' else -1)

In [7]:
paragraph_df = pd.read_csv("train_input_paragraph.csv")

In [8]:
paragraph_df #   SQuAD 2.0

,id,paragraph,theme
0,1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé
1,2,Following the disbandment of Destiny's Child i...,Beyoncé
2,3,"A self-described ""modern-day feminist"", Beyonc...",Beyoncé
3,4,"Beyoncé Giselle Knowles was born in Houston, T...",Beyoncé
4,5,Beyoncé attended St. Mary's Elementary School ...,Beyoncé
...,...,...,...
15550,15551,Isaac Newton (1643–1727) inherited Descartes' ...,Matter
15551,15552,"There is an entire literature concerning the ""...",Matter
15552,15553,In the late 19th century with the discovery of...,Matter
15553,15554,These quarks and leptons interact through four...,Matter


In [9]:
class NLPTransform(BasicTransform):

    """
    Base class for NLP transforms.  All NLP transforms should inherit from this class.

    Parameters:
    -----------
    always_apply : bool
        Apply the transform always, default: False

    p : float
        Probability of applying the transform, default: 0.5

    Inputs:
    -------
    data : text, language

    Image types:
    ------------
    text, language

    Returns:
    --------
    text, language
    
    """
    LANGS = {
        'en': 'english',
        'it': 'italian', 
        'fr': 'french', 
        'es': 'spanish',
        'tr': 'turkish', 
        'ru': 'russian',
        'pt': 'portuguese'
      }
      
    @property
    def targets(self) -> dict:
        """
        Mapping for data from class property names to `apply` function keywords.
        
        Returns:
            dict: {`property`: `keyword`}
            
        """
        return {"data": self.apply}
      
    def update_params(self, params, **kwargs) -> dict:
        """
        Update params with kwargs.
        
        Inputs:
        params (dict): 
            parameters to update.
                
        **kwargs:
            additional parameters to update.
            
        Returns:
            dict: updated parameters.
            
        """
        
        if hasattr(self, "interpolation"):
          params["interpolation"] = self.interpolation
        if hasattr(self, "fill_value"):
          params["fill_value"] = self.fill_value
        return params

    def get_sentences(self, text, lang='en') -> list:
        """ 
        Get sentences from text
        
        Inputs:
        -------
        text (str): 
            text to split into sentences
        lang (str): 
            language of text, default: 'en'
            
        Returns:
        --------
        list: 
            Tokenized list of sentences
                
        """

        return sent_tokenize(text, self.LANGS.get(lang, 'english'))

class ShuffleSentencesTransform(NLPTransform):
    """
    Takes a paragraph and shuffles the sentences in any order.  
      
    Parameters:
    -----------
    always_apply : bool         
        Apply the transform always, default: False
    p : float 
        Probability of applying the transform, default: 0.5
    
    Inputs:
    -------
    data : text, language
      
    Image types:
    ------------
    text, language
      
    Returns:
    --------
    text, language
    """
    def __init__(self, always_apply=False, p=0.5) -> None:
        """
        Do shuffle by sentence
        
        Parameters:
        -----------
        always_apply : bool
            Apply the transform always, default: False
        
        p : float
            Probability of applying the transform, default: 0.5
            
        Inputs:
        -------
        data : text, language
        
        Image types:
        ------------
        text, language
        
        Returns:
        --------
        None
        
        """
        super(ShuffleSentencesTransform, self).__init__(always_apply, p)
    
    def apply(self, data, **params) -> tuple:
        """
        Apply shuffle by sentence transform

        Parameters:
        -----------
        data : text, language

        Returns:
        --------
        text, language
        """
        text, lang = data
        sentences = self.get_sentences(text, lang)
        random.shuffle(sentences)
        return ' '.join(sentences), lang

In [10]:
import nlpaug.augmenter.word as naw

class WordAugmentation():
    """
    Word Augmentation class - performs word augmentation on a given dataframe containing paragraphs. 
    Takes in a dataframe of paragraphs, uses NLPAug library, Albumentations library and the NLPTransform class to perform word-level and sentence-level augmentation on the paragraphs.
    NLPAug library is used to perform word-level augmentations on GPU if GPU is available else on multiple CPU cores, while the NLPTransform class is used to perform sentence-level augmentation.
    The augmented paragraphs are then returned as a dataframe.
    For more information on the usage of NLPAug, please refer to https://github.com/makcedward/nlpaug
    For more information on the usage of Albumentations and NLPTransform, please refer to https://www.kaggle.com/code/shonenkov/nlp-albumentations 

    Inputs
    ----------
    paragraph_df : pd.DataFrame
        Dataframe containing the paragraphs to be augmented
    
    aug_per_paragraph : int, optional
        Number of augmentations to be performed per paragraph, by default 5 
        Must be of the form 4x+1 where x is an integer
    
    model_path : str, optional
        Path to the Model to be used for augmentation, must be a valid HuggingFace Transformer model - supports BERT, RoBERTa/LongFormer and BART models only, by default 'bert-base-uncased'
        For more information on the usage of HuggingFace Transformers, please refer to https://huggingface.co/transformers/

    device : str, optional
        Device to be used for augmentation, by default 'cpu'
        Must be either 'cpu' or 'cuda'
    
    lang : str, optional
        Language of the paragraphs, by default 'en'

    Class Variables
    ----------
    aug_sub : naw.ContextualWordEmbsAug
        Substitution augmentation object

    aug_insert : naw.ContextualWordEmbsAug
        Insertion augmentation object

    transform : NLPTransform.ShuffleSentencesTransform
        Sentence-level augmentation object

    paragraph_df : pd.DataFrame
        Dataframe containing the paragraphs to be augmented

    columns : list
        List of columns in the dataframe

    device : str
        Device to be used for augmentation
    
    cores : int
        Number of CPU cores to be used for augmentation

    lang : str
        Language of the paragraphs

    Methods
    -------
    augment()
        Performs word augmentation on the paragraphs and returns the augmented paragraphs as a dataframe

    Returns
    -------
    pd.DataFrame
        Dataframe containing the augmented paragraphs

    Example
    -------
    >>> from Data_Augmentation import WordAugmentation
    >>> aug = WordAugmentation(paragraph_df, aug_per_paragraph=5, model_path='bert-base-uncased', device='cpu', lang='en')
    >>> aug_df = aug.augment()

    """
    def __init__(
        self, 
        paragraph_df, 
        aug_per_paragraph = 5, 
        model_path = 'bert-base-uncased', 
        device='cpu',
        lang='en'
        ) -> None:

        """
        Constructor method

        Inputs
        ----------
        paragraph_df : pd.DataFrame
        Dataframe containing the paragraphs to be augmented
    
        aug_per_paragraph : int, optional
            Number of augmentations to be performed per paragraph, by default 5 
            Must be of the form 4x+1 where x is an integer
        
        model_path : str, optional
            Path to the Model to be used for augmentation, must be a valid HuggingFace Transformer model - supports BERT, RoBERTa/LongFormer and BART models only, by default 'bert-base-uncased'
            For more information on the usage of HuggingFace Transformers, please refer to https://huggingface.co/transformers/

        device : str, optional
            Device to be used for augmentation, by default 'cpu'
            Must be either 'cpu' or 'cuda'
        
        lang : str, optional
            Language of the paragraphs, by default 'en'

        Returns
        -------
        None

        """
        self.aug_sub = naw.ContextualWordEmbsAug(
            model_path= model_path, action="substitute", device=device)

        self.aug_insert = naw.ContextualWordEmbsAug(
            model_path= model_path, action="insert", device=device)

        self.device = device
        self.transform = ShuffleSentencesTransform(p=1.0)
        self.paragraph_df = paragraph_df
        self.columns = paragraph_df.columns
        self.lang = lang
        self.cores = mp.cpu_count()

        if (aug_per_paragraph-1) % 4 == 0:
            self.n = (aug_per_paragraph-1)/4
        else:
            raise ValueError("aug_per_paragraph must be of the form 1 + 4x")

    def __create_aug_df(self) -> None:

        """
        Creates a dataframe to store the augmented paragraphs
        
        Parameters
        ----------
        None

        Returns
        -------
        None

        """
        self.aug_df = pd.DataFrame(columns=self.columns)

    def __augment(self, paragraph) -> list:
        """
        Utility Function to perform word-level and sentence-level augmentation on a given paragraph

        Parameters
        ----------
        paragraph : str
            Paragraph to be augmented
        
        Returns
        -------
        list
            List containing the augmented paragraphs
    
        """
        aug_data = []

        if self.device == 'cpu':
            aug_first = self.aug_sub.augment(paragraph, n=self.n, num_thread=self.cores)            
            aug_second = self.aug_insert.augment(paragraph, n=self.n, num_thread=self.cores)
        else:
            aug_first = self.aug_sub.augment(paragraph, n=self.n)
            aug_second = self.aug_insert.augment(paragraph, n=self.n)    

        for i in aug_first:
            aug_data.append(i)
        for i in aug_second:
            aug_data.append(i)
        
        aug_data.append(self.transform(data=(paragraph, self.lang))['data'][0])
        
        start = int(2*self.n)
        end = int(4*self.n + 1)
        for i in range(start, end, 2):
            aug_data.append(self.transform(data=(aug_data[int(i/2 - 1)], self.lang))['data'][0])
        
        return aug_data

    def augment(self) -> pd.DataFrame:
        """
        Callable function to perform word-level and sentence-level augmentation on the paragraphs in the dataframe and stores in a new dataframe

        Parameters
        ----------
        None

        Returns
        -------
        pd.DataFrame
            Dataframe containing the augmented paragraphs
        
        Example
        -------
        >>> from Data_Augmentation import WordAugmentation
        >>> aug = WordAugmentation(paragraph_df, aug_per_paragraph=5, model_path='bert-base-uncased', device='cpu', lang='en')
        >>> aug_df = aug.augment()
        
        """

        self.__create_aug_df()

        for i in tqdm(range(len(self.paragraph_df)), description="Augmenting Paragraphs"):
            id = paragraph_df["id"][i]
            paragraph = paragraph_df["paragraph"][i]
            theme = paragraph_df["theme"][i]

            aug_data = self.__augment(paragraph)
            
            for j in range(len(aug_data)):
                self.aug_df = self.aug_df.append({"id": id, "paragraph": aug_data[j], "theme": theme}, ignore_index=True)
        
        return self.aug_df

In [ ]:
augmenter = WordAugmentation(paragraph_df, aug_per_paragraph=5, model_path='prajjwal1/bert-tiny', device='cpu', lang='en')
aug_paragraph_df = augmenter.augment()

In [ ]:
## Testing Code

# for i in tqdm(range(len(paragraph_df))):
#   id = paragraph_df["id"][i]
#   paragraph = paragraph_df["paragraph"][i]
#   theme = paragraph_df["theme"][i]
#   lang = 'en'
#   p1 = aug_sub.augment(paragraph, n = 1, num_thread = 2)
#   p2 = aug_insert.augment(paragraph, n= 1, num_thread = 2)
#   p3 = transform(data=(paragraph, lang))['data'][0]
#   aug_paragraph_df = aug_paragraph_df.append({"id": id, "paragraph": p1, "theme": theme}, ignore_index=True)
#   aug_paragraph_df = aug_paragraph_df.append({"id": id, "paragraph": p2, "theme": theme}, ignore_index=True)
#   aug_paragraph_df = aug_paragraph_df.append({"id": id, "paragraph": p3, "theme": theme}, ignore_index=True)
#   # aug_paragraph_df = aug_paragraph_df.append({"id": id, "paragraph": p4, "theme": theme}, ignore_index=True)

 45%|████▍     | 6939/15555 [46:58<1:11:55,  2.00it/s]

In [ ]:
aug_paragraph_df.head()

In [ ]:
aug_paragraph_df.to_csv()